In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [2]:
classes = {
    0: "default",
    1: "wave",
    2: "circle_cw",
    3: "cirlce_ccw",
    4: "rotate"
}

In [3]:


raw_data_file = "data/gyro_data.csv"
all_lines = []
with open(raw_data_file, 'r') as file:
    current_section = None
    section_row_index = None
    default_section_index = 0
    for line in file:
        data = line.strip().split(";")

        if len(data) == 3 and data[2] == "sample start":
            current_section = int(data[0])
            section_row_index = 0    

        if len(data) == 1 and data[0] == "sample end":
            current_section = None

        if current_section is not None and len(data) == 6:
            all_lines.append([section_row_index] + data + [current_section])
            section_row_index += 1
        elif len(data) == 6:
            all_lines.append([default_section_index] + data + [0])
            default_section_index += 1


df = pd.DataFrame(all_lines, columns=["ts", "AcX", "AcY", "AcZ", "GyX", "GyY", "GyZ", "label"])



In [ ]:
scaler = MinMaxScaler()
sensor_columns = ["AcX", "AcY", "AcZ", "GyX", "GyY", "GyZ"]
df[sensor_columns] = scaler.fit_transform(df[sensor_columns])
df

In [5]:
def plot_gyro_data(df: pd.DataFrame):
    for idx, name in classes.items():
        plt.figure(figsize=(20, 6))
        plt.title(f"Normalized Gyroscope data, class: {name}")
        class_data = df[df["label"] == idx]
        for column in sensor_columns:
            plt.plot(class_data["ts"], class_data[column], label=column)
        plt.legend()
        plt.show()

In [ ]:
plot_gyro_data(df)

In [ ]:
filtered_df = df[(df['ts'] >= 10) & (df['ts'] <= 320)]
plot_gyro_data(filtered_df)

In [8]:
train_test_cutoff = 250

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [10]:
features_train = filtered_df[filtered_df["ts"] < train_test_cutoff][['AcX', 'AcY', 'AcZ', 'GyX', 'GyY', 'GyZ']]
features_test = filtered_df[filtered_df["ts"] >= train_test_cutoff][['AcX', 'AcY', 'AcZ', 'GyX', 'GyY', 'GyZ']]
labels_train = filtered_df[filtered_df["ts"] < train_test_cutoff]["label"]
labels_test = filtered_df[filtered_df["ts"] >= train_test_cutoff]["label"]

In [ ]:
time_steps = 10
num_classes = 5

X_train = np.array([features_train[i - time_steps:i].values for i in range(time_steps, len(features_train))])
X_test = np.array([features_test[i - time_steps:i].values for i in range(time_steps, len(features_test))])

y_train = to_categorical(labels_train.values[time_steps:], num_classes=num_classes)
y_test = to_categorical(labels_test.values[time_steps:], num_classes=num_classes)
X_train.shape

In [ ]:

model = Sequential()
model.add(LSTM(units=10, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.save("../models/latest_model")

In [ ]:
model = tf.keras.models.load_model("../models/best_model")

In [ ]:
# Export weights so that they can be used in the c++ code
wandb = {
    "lstmIxWeights[60]": (0, 0, 10),
    "lstmFxWeights[60]": (0, 10, 10),
    "lstmCxWeights[60]": (0, 20, 10),
    "lstmOxWeights[60]": (0, 30, 10),
    "lstmIhWeights[100]": (1, 0, 10),
    "lstmFhWeights[100]": (1, 10, 10),
    "lstmChWeights[100]": (1, 20, 10),
    "lstmOhWeights[100]": (1, 30, 10),
    "lstmIBiases[10]": (2, 0, 10),
    "lstmFBiases[10]": (2, 10, 10),
    "lstmCBiases[10]": (2, 20, 10),
    "lstmOBiases[10]": (2, 30, 10),
    "denseLayerWeights[50]": (3, 0, 5),
    "denseLayerBiases[5]": (4, 0, 5)
}

weights = model.get_weights()
for key in wandb:
    layer, offset, length = wandb[key]
    print("const float {key} =  {{".format(key=key))
    if len(weights[layer].shape) == 2:
        data = weights[layer][:, offset:offset+length]

        if key != "dense_w":
            data = data.T
        for d in data:
            print("\t", end="")
            print(", ".join(map(str, d)), end=",\n")
    else:
        data = weights[layer][offset:offset+length]
        print("\t", end="")
        print(", ".join(map(str, data)))
    print("};")
    print()